<a href="https://colab.research.google.com/github/mankicom/DEV_GDPS_TEMP_LSTM/blob/master/Bayesian_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bayesian optimization을 이용한 TCN 초모수 최적화**
단기 풍속 편차보정모델 개발 예제



### **모듈로드**

In [ ]:
#-------------------------------------------------------------------------
# .. Module load

#.. module
import numpy as np
import pandas as pd
import os
import sys
from time import time

from sklearn.metrics import make_scorer, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
import joblib
import argparse
import f90nml
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

from tensorflow.compat.v1.keras.backend import set_session
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.models import model_from_json
from tensorflow.keras.layers import Dense, TimeDistributed
from tensorflow.keras import Input, Model, callbacks
from tensorflow.keras.utils import plot_model as plm
from tcn import TCN, tcn_full_summary
from tensorflow.keras.activations import swish


#.. local
sys.path.insert(0, './inc')
from tran_data_split import tran_data_split
from kmk_make_scorer import r2_3dim, mse_3dim, mae_3dim

### **ㅁㄴㅇㄹ**

In [ ]:
#-------------------------------------------------------------------------
# .. Device configuration

# .. Set number of gpu
#ap = argparse.ArgumentParser()
#ap.add_argument("-g", "--gpus", type=int, default=1,
#                help="# of GPUS to use for training")
#args = vars(ap.parse_args())
#G = args["gpus"]
#
#print("[INFO] {} cores available".format(G))



# .. Memory set up
#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
##config.gpu_options.per_process_gpu_memory_fraction = 0.4
#K.set_session(tf.Session(config=config))

#seed = 1
#np.random.seed(seed)


config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.compat.v1.Session(config=config))

### **ㅁㄴㅇㄹ**

In [ ]:
#-------------------------------------------------------------------------
# .. Data set

element = 'ALLV'
name_list = "./SHEL/namelist.input"

hp_lr = 0.009
hp_pd = 'same'
hp_ns = 1
hp_dl = [1,2,4,8,17,34,68,136]

n_iter_search = 20

### **ㅁㄴㅇㄹ**

In [ ]:
#-------------------------------------------------------------------------
# .. Read namelist

print ("1. Read namelist")
exists = os.path.isfile(name_list)
if exists:
    nml = f90nml.read(name_list)
    tran_data_per = nml['data_set']['tran_data_per']
    tran_num_his = nml['data_set']['tran_num_his']
    test_data_per = nml['data_set']['test_data_per']
    test_num_his = nml['data_set']['test_num_his']
    num_fct = nml['data_set']['num_fct']
    dev_stn_id = nml['data_set']['dev_stn_id']
    exp_name = nml['data_set']['exp_name']
    data_dir = nml['data_set']['data_dir']
    input_size = nml['data_set']['input_size']
    output_size = nml['data_set']['output_size']
    num_epoch = nml['hyper_para']['num_epoch']
    patience = nml['hyper_para']['patience']
    hp_nf = nml['hyper_para']['n_filter']
    hp_ks = nml['hyper_para']['s_kernel']
    hp_dr = nml['hyper_para']['drp_rate']
else:
    sys.exit("STOP Error: Could not found : "+ name_list)

hp_ldl = hp_dl[-1] # last dilation factor to make name of save model
hp_bn = True


#data_dir = './DAIN_MIX/'
#data_dir = './DAIN_HR136/'
csv_outdir = './DAOU/LOSS/' + exp_name + '/'
model_outdir = './DAOU/MODL/' + exp_name + '/'
scalr_outdir = './DAOU/SCAL/' + exp_name + '/'
gifd_outdir = './GIFD/' + exp_name + '/'

### **ㅁㄴㅇㄹ**

In [ ]:
#-------------------------------------------------------------------------
# .. Fcst load :  data dim( input_size, num_stn, num_his, num_fct )
#
#    Trainining data used for cross-validation
#    Test data used to evaluate best model out of randomized search
#

print ("3. Training/valid data load, combine 4stn(47169, 47133, 47102, 47090) train sample")
tran_rate = 0.8
eval_rate = 0.2
rd_seed_fix = False
nbin = 10

combine_stn = [47169, 47133, 47102, 47090]
for i in range(len(combine_stn)):
    tran_xx, tran_yy, test_xx, test_yy = tran_data_split(data_dir, tran_data_per,
                                     element, input_size, output_size, tran_num_his,
                                     num_fct, combine_stn[i], tran_rate, nbin, rd_seed_fix)

    if i == 0:
       tran_x, tran_y, test_x, test_y = tran_xx, tran_yy, test_xx, test_yy
    else:
       tran_x = np.concatenate((tran_x,tran_xx), axis=0)
       tran_y = np.concatenate((tran_y,tran_yy), axis=0)
       test_x = np.concatenate((test_x,test_xx), axis=0)
       test_y = np.concatenate((test_y,test_yy), axis=0)

### **ㅁㄴㅇㄹ**

In [ ]:
#-------------------------------------------------------------------------
# .. Select variable


input_size = tran_x.shape[2]
output_size = tran_y.shape[2]
print ("5. Select var")
print ('tran_x shape= ', tran_x.shape)    # batch, sequence, feature
print ('tran_y shape= ', tran_y.shape)
print ('test_x shape= ', test_x.shape)    # batch, sequence, feature
print ('test_y shape= ', test_y.shape)


### **ㅁㄴㅇㄹ**

In [ ]:
#-------------------------------------------------------------------------
# .. Normalize

# .. initialaize
tr_b, tr_s, tr_f = tran_x.shape[0], tran_x.shape[1], tran_x.shape[2]
ts_b, ts_s, ts_f = test_x.shape[0], test_x.shape[1], test_x.shape[2]

# .. get restorator with obs range
nwp_scaler = MinMaxScaler()   # copy default true
obs_scaler = MinMaxScaler()
nwp_scaler.fit(tran_x.view().reshape(tr_b*tr_s, tr_f))
obs_scaler.fit(tran_y.view().reshape(tr_b*tr_s, output_size))

# .. feature normalize   ( train seq, feature = test seq, feature )
nor_tran_x = nwp_scaler.transform(tran_x.reshape(tr_b*tr_s, tr_f))
nor_tran_x = nor_tran_x.reshape(tr_b,tr_s,tr_f)
nor_tran_y = obs_scaler.transform(tran_y.reshape(tr_b*tr_s, output_size))
nor_tran_y = nor_tran_y.reshape(tr_b,tr_s, output_size)

nor_test_x = nwp_scaler.transform(test_x.reshape(ts_b*ts_s, ts_f))
nor_test_x = nor_test_x.reshape(ts_b,ts_s,ts_f)
nor_test_y = obs_scaler.transform(test_y.reshape(ts_b*ts_s, output_size))
nor_test_y = nor_test_y.reshape(ts_b,ts_s, output_size)


print ('---------- Final training data shape')
print(type(nor_tran_x))
print ('tran nwp : ', nor_tran_x.shape)
print ('tran obs : ', nor_tran_y.shape)
print ('test nwp : ', nor_test_x.shape)
print ('test obs : ', nor_test_y.shape)

### **ㅁㄴㅇㄹ**

In [ ]:

#=========================================================================
# .. Model configuration



#-------------------------------------------------------------------------
# .. Set batch for cross-validation

if eval_rate == 0.2: num_cv=5
#if eval_rate == 0.2: num_cv=2
real_train_size = nor_tran_x.shape[0]
batch_size = int( real_train_size*(1-eval_rate)*0.1 )
#batch_size = real_train_size
#batch_size = 572

print ('input_size: ', input_size)
print ('batch_size: ', batch_size)
print ('time_lenght: ', num_fct)

### **ㅁㄴㅇㄹ**

In [ ]:
#-------------------------------------------------------------------------
# .. Set Model


# .. Define model
def create_model(dropout_rate=0.15, nb_filters=7, kernel_size=3):

          print ('================== Model called ========================')
          print ('input_size: ', input_size)
          print ('batch_size: ', batch_size)
          print ('time_lenght: ', num_fct)
          print ('nb_filters: ', nb_filters)
          print ('kernel_size: ', kernel_size)
          print ('dropout_rate: ', dropout_rate)
          print ('dilations: ', hp_dl)
          dropout_rate = np.round(dropout_rate,2)
          print ('dropout_rate: ', dropout_rate)

          ## .. clear keras model
          K.clear_session()

          # .. create model
          #i = Input( batch_shape=(batch_size, num_fct, input_size) )
          i = Input( batch_shape=(None, num_fct, input_size) )
          o = TCN(return_sequences=True,
                  activation=swish,
                  nb_filters=nb_filters,
                  padding=hp_pd,
                  use_batch_norm = hp_bn,
                  nb_stacks=hp_ns,
                  dropout_rate=dropout_rate,
                  kernel_size=kernel_size,
                  use_skip_connections=True,
                  dilations=hp_dl
                  )(i)
          o = TimeDistributed(Dense(output_size, activation='linear'))(o)

          # .. compile
          adam = optimizers.Adam(lr=hp_lr)

          return m


# .. Wrapping create_model for searching hyper-parameter
model = KerasRegressor(build_fn=create_model,
                       verbose=1,
                       epochs=num_epoch,
                       batch_size=batch_size,
                       shuffle=True)

### **ㅁㄴㅇㄹ**

In [ ]:
#-------------------------------------------------------------------------
# .. Use bayes_opt

# .. Exp para set
#param_dist = { 'padding': Categorical(['causal','same']),
#               'nb_stacks': Integer(1,5),
#               'nb_filters': Categorical([7,20,30]),
#               'kernel_size': Integer(2,24) }
#param_dist = { 'dropout_rate': Real(0.01, 0.2),
#               'nb_filters': Integer(50,100),
#               'kernel_size': Integer(3,12) }
##param_dist = { 'dropout_rate': Categorical([0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.15]),
#               'nb_filters': Integer(50,100),
#               'kernel_size': Integer(3,12) }
param_dist = { 'nb_filters': Integer(50,100),
               'kernel_size': Integer(3,12) }

set_eval_score = { 'MAE': make_scorer(mae_3dim),
                   'MSE': make_scorer(mse_3dim),
                   'R2': make_scorer(r2_3dim) }

                   print ( param_dist )

optimizer =  BayesSearchCV( estimator=model,
                            search_spaces=param_dist,
                            scoring=make_scorer(r2_3dim),
                            refit=False,
                            cv=num_cv,
                            n_iter=n_iter_search,
                            return_train_score=True,
                            verbose=1,
                            n_jobs=1,
                            random_state=1 )

start = time()
print(nor_tran_x.shape, nor_tran_y.shape)
optimizer.fit(nor_tran_x, nor_tran_y)

print(type(optimizer.cv_results_))
print(optimizer.cv_results_)

# .. Report
def report(result, n_top=n_iter_search):
    for i in range(n_top):
        candidates = [ result['rank_test_score'][i] ]
        for candidate in candidates:
            print("Rank: %0d, R2: %.3f with %r" %
                  ( i, result['mean_test_score'][candidate-1],
                       result['params'][candidate-1] ) )


print("BayesSearchCV took %.2f seconds for %d candidates"
      " parameter settings. " % ((time() - start), n_iter_search))

print( "Best: %f using %s" % ( optimizer.best_score_,
                               optimizer.best_params_ ) )

report(optimizer.cv_results_)

### **ㅁㄴㅇㄹ**

In [ ]:
#=========================================================================
# .. Second refit to evaluate best model use test period


#-------------------------------------------------------------------------
# .. Set model label

# .. best model configuration for whole train set
params = optimizer.best_params_
print(type(params))
print(params)